In [2]:
import os   # 디렉토리 경로 호출 용도 
import cv2   # 이미지 파일 불러올 때 사용
import numpy as np  # 다양한 행렬 연산 (데이터 처리) 용도
from sklearn.preprocessing import LabelEncoder  
# 데이터 전처리 (문자로된 폴더 리스트를 숫자형 array로 변환)
from sklearn.preprocessing import OneHotEncoder
#one-hot-encoding을 위해 OneHotEncoder 함수를 불러옴
from numpy import array # 리스트를 array형태로 만들떄 사용하는 함수
import tensorflow as tf

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

tf.set_random_seed(1)

# 학습한 모델을 불러오기 위해 그래프를 만들어줌

In [3]:
tf.reset_default_graph()  # 이전에 생성해둔 graph, variable 모두 초기된다.

TRAIN_DIR = 'E:\\data\\vinzip\\trainingSet\\'
train_folder_list = array(os.listdir(TRAIN_DIR))
train_input = []
train_label = []
 
label_encoder = LabelEncoder()  # LabelEncoder Class 호출

integer_encoded = label_encoder.fit_transform(train_folder_list)  ###[0 1 2]

#문자열로 구성된 train_folder_list를 숫자형 리스트로 변환

onehot_encoder = OneHotEncoder(sparse=False) 
#print(onehot_encoder)
#OneHotEncoder(categorical_features=None, categories=None,dtype=<class 'numpy.float64'>, handle_unknown='error', n_values=None, sparse=False)

integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)  ## [[0][1][2]]

onehot_encoded = onehot_encoder.fit_transform(integer_encoded)   ## [[1. 0. 0.] [0. 1. 0.] [0. 0. 1.]]


for index in range(len(train_folder_list)):
    path = os.path.join(TRAIN_DIR, train_folder_list[index])
    path = path + '\\' 
    img_list = os.listdir(path) # <- img_1.jpg img_2.jpg....
    for img in img_list:
        img_path = os.path.join(path, img) # E:\data\MNIST\trainingSet\0_zero\ + img_1.jpg 
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # 이미지를 불러옴
        train_input.append([np.array(img)]) # 불러온 이미지를 array로 바꿔서 append
        train_label.append([np.array(onehot_encoded[index])]) # one_hot 한 라벨도 append

train_input = np.reshape(train_input, (-1,4096))  # (3800, 4096)
train_label = np.reshape(train_label, (-1, 3))  # (3800, 3)

train_input = np.array(train_input).astype(np.float32) # (3800, 4096) => 2차원 배열로 배열안에 각각 이미지가 행렬 형태로 저장됨
train_label = np.array(train_label).astype(np.float32) # (3800,3) => 2차원 배열로 배열안에 라벨 값이 행렬 형태로 저장됨 [[0 0 1] [1 0 0 ]..]

TEST_DIR ='E:\\data\\vinzip\\testSet\\'
test_folder_list = array(os.listdir(TEST_DIR))
 
test_input = []
test_label = []
 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(test_folder_list)
 
onehot_encoder = OneHotEncoder(sparse=False) 
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
 
for index in range(len(test_folder_list)):
    path = os.path.join(TEST_DIR, test_folder_list[index])
    path = path + '\\'
    img_list = os.listdir(path)
    for img in img_list:
        img_path = os.path.join(path, img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        test_input.append([np.array(img)])
        test_label.append([np.array(onehot_encoded[index])])
 
test_input = np.reshape(test_input, (-1, 4096))
test_label = np.reshape(test_label, (-1, 3))
test_input = np.array(test_input).astype(np.float32)
test_label = np.array(test_label).astype(np.float32)
# np.save("test_input.npy",test_input)
# np.save("test_label.npy",test_label)
 
# hyper parameters
learning_rate = 0.001
 
# input place holders
X = tf.placeholder(tf.float32, [None, 4096]) 
X_img = tf.reshape(X, [-1, 64, 64, 1])   # img 28x28x1 (black/white)

Y = tf.placeholder(tf.float32, [None, 3])

keep_prob = tf.placeholder(tf.float32)

 
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))

L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
# [64,64,1] --> [64,64,32] (32개의 featuremap을 가짐)

L1 = tf.nn.relu(L1) # 활성화 함수는 렐루를 사용
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# [64,64,32] --> [32,32,32] (2x2 max_poo;ing을 거침)

#L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#L2 = tf.nn.dropout(L2, keep_prob)  #안쓰는게 더 정확함 


W3 = tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob)

L3_flat = tf.reshape(L3, [-1, 8 * 8 * 64]) # 다시 평평하게

W4 = tf.get_variable("W4", shape=[8 * 8 * 64, 3], initializer=tf.contrib.layers.xavier_initializer())
                 
b = tf.Variable(tf.random_normal([3]))
#bias를 선언하여 b에 저장한다. output의 shape이 10이므로 shape을 10으로 설정
logits = tf.matmul(L3_flat, W4) + b

# define cost/loss &amp;amp;amp; optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
 #cost를 최소화하는 optimizer 함수를 선언
    
training_epochs = 100

batch_size = 600


C:\Users\pc\Anaconda3\envs\zz\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\pc\Anaconda3\envs\zz\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In c

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# 이미지를 학습 시켰던 사이즈로 변환 (원본은 그대로)

In [4]:
def re_size_img(path):
    
    img_size = 64
    #path = '.\\vinzip_test\\'
    dir_name = 'resize_img'
    
    img_type = '\\*.jpg'
    
    try:
        os.mkdir(path+dir_name)
    except Exception as e:
        #print(e)
        pass
    
    imglist = glob.glob(path+img_type)

    ## 이미지 사이즈 조절
    for img_p in imglist:
        img = Image.open(img_p)
        img.resize((img_size,img_size)).save(path+dir_name+'\\'+img_p[(len(path)):])
    
    return imglist
#re_size_img()

# 학습시킨 모델에 이미지를 넣어 라벨값을 반환받음

In [5]:
def model_return(TEST_DIR2):

    #TEST_DIR2 ='E:\\data\\vinzip\\testSet2\\'
    TEST_DIR2 = TEST_DIR2 + 'resize_img'

    img_type = '\\*.jpg'
    test_folder_list2 = glob.glob(TEST_DIR2+img_type)
    
    test_input2 = []
    test_label2 = []

    for index in range(len(test_folder_list2)):
        path = os.path.join(TEST_DIR2, test_folder_list2[index])
        print(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        test_input2.append([np.array(img)])

    test_input2 = np.reshape(test_input2, (-1, 4096))
    test_input2 = np.array(test_input2).astype(np.float32)
    test_label2 = np.array(test_label2).astype(np.float32)


    save_file = './train_model.ckpt'
    saver = tf.train.Saver()
    
    out_lst=[]
    # Launch the graph
    with tf.Session() as sess:
        saver.restore(sess, save_file)

        print(sess.run(tf.argmax(logits, 1),feed_dict={X: test_input2, keep_prob: 1}))
        out_lst = sess.run(tf.argmax(logits, 1),feed_dict={X: test_input2, keep_prob: 1})

    try:
        shutil.rmtree(TEST_DIR2)
    except OSError as e:
        if e.errno == 2:
        # 파일이나 디렉토리가 없음!
            print('No such file or directory to remove')
            pass
        else:
            print(e)

    return out_lst

# 각 라벨에 맞는 디렉토리로 이미지 이동

In [6]:
from os import rename
import glob
import cv2
import os
from os import remove
from PIL import Image
import sys
import shutil


TEST_DIR2 ='E:\\data\\vinzip\\testSet2\\'
dir_name = 'classification_result\\'
img = re_size_img(TEST_DIR2)
out_lst =  model_return(TEST_DIR2)

try:
    os.mkdir(TEST_DIR2+dir_name)
except Exception as e:
    pass


## 디렉토리로 종류 분류 
for i,v in enumerate(out_lst):
    if v ==0:
        try:
            os.mkdir(TEST_DIR2+dir_name+'bottom')
        except Exception as e:
            pass
        rename(img[i],TEST_DIR2+dir_name+'bottom\\'+img[i][len(TEST_DIR2):])

    elif v ==1:
        try:
            os.mkdir(TEST_DIR2+dir_name+'outer')
        except Exception as e:
            pass
        rename(img[i],TEST_DIR2+dir_name+'outer\\'+img[i][len(TEST_DIR2):])

    elif v ==2:
        try:
            os.mkdir(TEST_DIR2+dir_name+'top')
        except Exception as e:
            pass
        rename(img[i],TEST_DIR2+dir_name+'top\\'+img[i][len(TEST_DIR2):])
        
print('===# classification finish #===')

E:\data\vinzip\testSet2\resize_img\1568.jpg
E:\data\vinzip\testSet2\resize_img\1569.jpg
E:\data\vinzip\testSet2\resize_img\1570.jpg
E:\data\vinzip\testSet2\resize_img\1571.jpg
E:\data\vinzip\testSet2\resize_img\1572.jpg
E:\data\vinzip\testSet2\resize_img\1573.jpg
E:\data\vinzip\testSet2\resize_img\1574.jpg
E:\data\vinzip\testSet2\resize_img\1575.jpg
E:\data\vinzip\testSet2\resize_img\1576.jpg
E:\data\vinzip\testSet2\resize_img\1577.jpg
E:\data\vinzip\testSet2\resize_img\1578.jpg
E:\data\vinzip\testSet2\resize_img\1579.jpg
E:\data\vinzip\testSet2\resize_img\1580.jpg
E:\data\vinzip\testSet2\resize_img\1581.jpg
E:\data\vinzip\testSet2\resize_img\1582.jpg
E:\data\vinzip\testSet2\resize_img\1583.jpg
E:\data\vinzip\testSet2\resize_img\1584.jpg
E:\data\vinzip\testSet2\resize_img\1585.jpg
E:\data\vinzip\testSet2\resize_img\1586.jpg
E:\data\vinzip\testSet2\resize_img\1587.jpg
E:\data\vinzip\testSet2\resize_img\1588.jpg
E:\data\vinzip\testSet2\resize_img\1589.jpg
E:\data\vinzip\testSet2\resize_i